In [1]:
import urllib
import requests
import itertools
from copy import deepcopy as mcopy
from music21 import *

from ksxml import permute, \
    nth_permutation_indices, \
    send_m21_object, \
    write_midi_file, \
    send_transport_command

In [5]:
send_transport_command('a', 'play')

<Response [200]>

In [2]:
def hashtag_music(note_names):
    midi_numbers   = [note.Note(name).pitch.midi for name in note_names]
    with_octave_up = midi_numbers + [number + 12 for number in midi_numbers]
    appendage      = with_octave_up[1:-1]
    appendage.reverse()
    forward_back   = with_octave_up + appendage
    note_list = [note.Note(n, type='16th') for n in forward_back]
    
    # When making a note from a midi number, m21 automagically inserts a 'natural' accidental
    for n in note_list:
        if n.pitch.accidental and n.pitch.accidental.name == 'natural':
            n.pitch.accidental = None
    
    s1 = stream.Part()        
    s1.append(note_list)
    return s1

In [137]:
# source material
n1 = ['C3', 'D3', 'G3', 'A3']
n2 = ['C3', 'e3', 'G3', 'b3'] 
n3 = ['D3', 'E3', 'A3', 'B3']

chords_with_names = [n1, n1, n2, n2, n3, n3]

In [12]:
# Permute, then mirror
permutation_stream = stream.Stream()
for notes_midi, order, r in zip(itertools.cycle(cycle_me), permute(list(range(8))), range(200)):
    
    re_ordered_midi = [notes_midi[i] for i in order]

    # ramp arp back down
    reversed_midi = re_ordered_midi[1: -1]
    reversed_midi.reverse()    
    forward_back_midi = re_ordered_midi + reversed_midi
        
    # add 
    permutation_stream.append([note.Note(num, type='16th') for num in forward_back_midi])

In [138]:
def note_list_from_note_names(note_names):
    notes = [note.Note(name, type='16th') for name in note_names]
    
    # When making a note from a midi number, m21 automagically inserts a 'natural' accidental
    for n in notes:
        if n.pitch.accidental and n.pitch.accidental.name == 'natural':
            n.pitch.accidental = None
    return notes

def order_generator(size, start_index, max_steps):
    for i in range(start_index, start_index + max_steps):
        yield nth_permutation_indices(i, size)
        
def hashtagify_chord_arrays(note_name_arrays):
    midi_number_arrays = [[note.Note(name).pitch.midi for name in note_names] for note_names in note_name_arrays]
    midi_number_arrays = [ns + [n + 12 for n in ns] for ns in midi_number_arrays]
    results = []
    for midi_array in midi_number_arrays:
        appendage = midi_array[1:-1]
        appendage.reverse()
        results.append(midi_array + appendage)
    return results

def stream_generator_from_chord_arrays(chords, start_index, count):
    chords = hashtagify_chord_arrays(chords)
    length = len(chords[0])

    # Mirror, then permute
    for midi_notes, order in zip(itertools.cycle(chords), order_generator(length, start_index, count)):
        midi_melody = [midi_notes[i] for i in order]
        note_melody = note_list_from_note_names(midi_melody)
        
        melody_stream = stream.Stream()
        melody_stream.append(note_melody)
        yield melody_stream



In [141]:
stream_gen = stream_generator_from_chord_arrays(chords_with_names, 0, 400)
streams = [s for s in stream_gen]
full_stream = stream.Stream()
for s in streams: full_stream.append(s)
write_midi_file(full_stream.flat, '/session/default.mid')

In [151]:
send_transport_command('a', 'play')
for s in streams:
    s.insert(clef.BassClef())
    send_m21_object('a', s)

KeyboardInterrupt: 

In [24]:
# Test pushing score to browser client from a m21 stream object
stream1 = hashtag_music(n1)
send_m21_object('a', stream1)

<Response [200]>

In [39]:
# Test writing a score file
write_midi_file(permutation_stream, '/session/default.mid')

In [33]:
# Test Transport Controls
send_transport_command('a', 'play')

<Response [200]>

In [136]:
send_transport_command('a', 'stop')

<Response [200]>

In [40]:
send_transport_command('a', 'reload')

<Response [200]>

15